In [1]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments)
from trl import SFTTrainer

C:\Users\BhaskerSriHarsha\Documents\Environments\pyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the data

In [2]:
import pickle as pkl

In [3]:
train_data = pkl.load(open("Dataset/ChatTrain.pkl","rb"))
val_data = pkl.load(open("Dataset/ChatTest.pkl","rb"))

train_data, val_data

(Dataset({
     features: ['text'],
     num_rows: 8625
 }),
 Dataset({
     features: ['text'],
     num_rows: 455
 }))

## Load bits and bytes config

In [4]:
bnb_4bit_compute_dtype = "float16"
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,)

## Load the model and tokenizer

In [5]:
model_name = "ibm-granite/granite-3.3-2b-base"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.83s/it]


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## Setup LoRA config

In [8]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"] )

In [9]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

## Setup SFT Trainer

In [10]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=50,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none")

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    args=training_arguments,)

C:\Users\BhaskerSriHarsha\Documents\Environments\pyTorch\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\BhaskerSriHarsha\Documents\Environments\pyTorch\Lib\site-packages\peft\mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'ibm-granite/granite-3.3-2b-base' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
C:\Users\BhaskerSriHarsha\Documents\Environments\pyTorch\Lib\site-packages\peft\tuners\tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Truncating train dataset: 100%|█████████████████████████████████████████| 8625/8625 [00:00<00:00, 831954.37 

In [12]:
trainer.train()

C:\Users\BhaskerSriHarsha\Documents\Environments\pyTorch\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,0.721100
100,0.570800
150,0.567300
200,0.552500
250,0.552500


C:\Users\BhaskerSriHarsha\Documents\Environments\pyTorch\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\BhaskerSriHarsha\Documents\Environments\pyTorch\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*

TrainOutput(global_step=270, training_loss=0.5862758530510797, metrics={'train_runtime': 4211.1922, 'train_samples_per_second': 2.048, 'train_steps_per_second': 0.064, 'total_flos': 9331275494965248.0, 'train_loss': 0.5862758530510797})

In [13]:
trainer.model.save_pretrained("Ganite-2b-finetuned")